In [2]:
%%HTML

<script>
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                jupyter.actions.call('jupyter-notebook:run-cell');
            });
        }
    );
</script>

<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>

In [3]:
from ipywidgets import widgets
from IPython.display import display

In [4]:
defaultRate = .2999
timeFrameMonths = 48

weightAPR = .7
weightCashback = .15
weightFees = .1
weightGracePeriod = .05

def CreditCardScore(card):
    if (card == 'CitiForward'):
        apr_Standard_Purchase = .1724
        apr_Standard_CashAdvance = .2199
        apr_Promo_Purchase_6Months1 = apr_Standard_Purchase
        apr_Promo_Purchase_6Months2 = apr_Standard_Purchase
        apr_Promo_Purchase_6Months3 = apr_Standard_Purchase
        apr_Promo_Purchase = apr_Standard_Purchase
        apr_APRReduction = .0025
        Cashback_General_Bracket1 = 0
        Cashback_General_Bracket1_Max = 0
        Cashback_General_Bracket2 = 0
        Cashback_Categorical_YearlyLimit = 750
        fee_BalanceTransfer = .03
        fee_CashAdvance = .03
        fee_ForeignPurchase = .03
        fee_Late_Bracket1_Amount = 15
        fee_Late_Bracket2_Amount = 29
        fee_Late_Bracket3_Amount = 39
        fee_OverCreditLine = 39
        goodCreditScoreNeeded = True
        gracePeriodDays = 0

    elif (card=='CitiPlatinum'):
        apr_Standard_Purchase = .2124
        apr_Standard_CashAdvance = .2199
        apr_Promo_Purchase_6Months1 = 0
        apr_Promo_Purchase_6Months2 = .1724
        apr_Promo_Purchase_6Months3 = .1824
        apr_APRReduction = 0
        Cashback_General_Bracket1 = 0
        Cashback_General_Bracket1_Max = 0
        Cashback_General_Bracket2 = 0
        Cashback_Categorical_YearlyLimit = 750
        fee_BalanceTransfer = .03
        fee_CashAdvance = .03
        fee_ForeignPurchase = .03
        fee_Late_Bracket1_Amount = 15
        fee_Late_Bracket2_Amount = 29
        fee_Late_Bracket3_Amount = 39
        fee_OverCreditLine = 39
        goodCreditScoreNeeded = True
        gracePeriodDays = 0

    else: #(card='Discover')
        apr_Standard_Purchase = .1499
        apr_Standard_CashAdvance = .2399
        apr_Promo_Purchase_6Months1 = 0
        apr_Promo_Purchase_6Months2 = .1499
        apr_Promo_Purchase_6Months3 = .1499
        apr_APRReduction = 0
        Cashback_General_Bracket1 = .0025
        Cashback_General_Bracket1_Max = 3000
        Cashback_General_Bracket2 = .01
        Cashback_Categorical_YearlyLimit = 6000
        fee_BalanceTransfer = 0
        fee_CashAdvance = .05
        fee_ForeignPurchase = .02
        fee_Late_Bracket1_Amount = 0
        fee_Late_Bracket2_Amount = 29
        fee_Late_Bracket3_Amount = 39
        fee_OverCreditLine = 0
        goodCreditScoreNeeded = False
        gracePeriodDays = 25
  
    apr_Standard_Purchase_Predict_Reduction = (apr_APRReduction*(min((indicatorResults.result[3]/850),(indicatorResults.result[5]/10))))*8
    apr_Standard_Purchase_Predict = apr_Standard_Purchase+((1-(min((indicatorResults.result[3]/850),(indicatorResults.result[5]/10))))*(defaultRate-apr_Standard_Purchase))-apr_Standard_Purchase_Predict_Reduction
    
    score_APR_Purchase = ((((min((indicatorResults.result[3]/850),(indicatorResults.result[5]/10)))*(((min(6,timeFrameMonths))*((defaultRate-apr_Promo_Purchase_6Months1)/defaultRate)) + ((max(0,(min(6,(timeFrameMonths-6)))))*((defaultRate-apr_Promo_Purchase_6Months2)/defaultRate)) + ((max(0,(min(6,(timeFrameMonths-12)))))*((defaultRate-apr_Promo_Purchase_6Months3)/defaultRate)))) + ((max(0,(timeFrameMonths-12)))*((defaultRate-apr_Standard_Purchase_Predict)/defaultRate)))*100)/timeFrameMonths
    score_APR_CashAdvance = ((defaultRate-apr_Standard_CashAdvance)/defaultRate)*100
    
    score_Cashback_General = (((min(indicatorResults.result[6],3000))*Cashback_General_Bracket1)+((max((indicatorResults.result[6]-Cashback_General_Bracket1_Max),0))*Cashback_General_Bracket2))/200*80
    if (indicatorResults.result[4] == 4):
        bonusThankYouMoney = 40
    else:
        bonusThankYouMoney = 0
    if (card=='Discover'):
        score_Cashback_Categorical = ((Cashback_Categorical_YearlyLimit*(indicatorResults.result[0]/10))/60)
    else:
        score_Cashback_Categorical = (((Cashback_Categorical_YearlyLimit*(indicatorResults.result[1]/10))+bonusThankYouMoney)/60)
    
    score_Fees = 160-((1000*(fee_BalanceTransfer+fee_CashAdvance+(fee_ForeignPurchase*(indicatorResults.result[2]/10))))+((fee_Late_Bracket1_Amount+fee_Late_Bracket2_Amount+fee_Late_Bracket3_Amount)/3)+fee_OverCreditLine)
    
    score_GracePeriod = (gracePeriodDays/31)*100
    
    if ((goodCreditScoreNeeded is True) and (indicatorResults.result[3]<700)):
        score_Total = 'Credit Score Too Low'
    else:
        score_Total = round((weightAPR*(score_APR_Purchase+score_APR_CashAdvance))+(weightCashback*(score_Cashback_General+score_Cashback_Categorical))+(weightFees*score_Fees)+(weightGracePeriod*score_GracePeriod))
    
    return score_Total

In [5]:
from IPython.display import display
def Slider(indicator_Bonus_Discover,indicator_Bonus_Citi,indicator_TravelFrequency,indicator_CreditHistory,indicator_Bonus_CitiPlatinum,indicator_Punctuality,indicator_Spending):
    return(indicator_Bonus_Discover,indicator_Bonus_Citi,indicator_TravelFrequency,indicator_CreditHistory,indicator_Bonus_CitiPlatinum,indicator_Punctuality,indicator_Spending)
indicatorResults = widgets.interactive(
    Slider,
    indicator_Bonus_Discover=widgets.IntSlider(description='Frequency of Purchase: Home Improvement/Department/Pet Stores and Gas',min=1,max=10),
    indicator_Bonus_Citi=widgets.IntSlider(description='Frequency of Purchase: Record/Video Rental Stores and Theaters',min=1,max=10),
    indicator_TravelFrequency=widgets.IntSlider(description='Foreign Travel Frequency',min=1,max=10),
    indicator_CreditHistory=widgets.IntSlider(description='Credit Score',min=300,max=850),
    indicator_Bonus_CitiPlatinum=widgets.IntSlider(description='Typical GPA',min=1,max=4),
    indicator_Punctuality=widgets.IntSlider(description='Punctuality',min=1,max=10),
    indicator_Spending=widgets.IntSlider(description='Spent/Yr',min=1,max=20000)
)

def ButtonClick(b):
    print('CitiPlatinum:',CreditCardScore('CitiPlatinum'))
    print('CitiForward:',CreditCardScore('CitiForward'))
    print('Discover:',CreditCardScore('Discover'))
    print('')

button = widgets.Button(description="Calculate")

display(indicatorResults)
display(button)

button.on_click(ButtonClick)

(1, 1, 1, 300, 1, 1, 1)